#Example how to import data from kaggle to colab

In [0]:
#@title Import Dependencies and upload kaggle.json
from google.colab import files
!pip install kaggle
files.upload()


In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/


In [0]:
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
# Print the competition list
!kaggle competitions list

In [0]:
!kaggle competitions download -c ultrasound-nerve-segmentation

In [0]:
!unzip test.zip

In [0]:
!unzip train.zip

In [0]:
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
import os


In [0]:
def create_data():
  
  ''' The function generates the train data and the test data
      if os.getcwd ---> train folder the function returns X_train, Y_train
      if os.getcwd ---> test folder the function returns X_test, names
      names are the file names
  '''
  
  total = len(os.listdir())
  img_rows = 420
  img_cols = 580
  k=0
  
  if ('_' in os.listdir()[0]) :
    
    X_train = np.ndarray((total//2,img_rows,img_cols), dtype = np.uint8)
    Y_train = np.ndarray( (total//2,img_rows,img_cols), dtype = np.uint8)
  
    for i in os.listdir():
      if 'mask' not in i:
        
        X_train[k] = np.array([imread(i, as_grey = True)])    
        Y_train[k] = np.array(imread(i.split('.')[0]+'_mask.tif', 
                                     as_grey = True))
        k+=1
    np.save('/content/imgs_train.npy', X_train)
    np.save('/content/imgs_mask_train.npy', Y_train)
    return X_train, Y_train

  else:
    names = []
    X_test = np.ndarray( (total,img_rows,img_cols), dtype = np.uint8)
    for i in os.listdir():      
        X_test[k] = np.array(imread(i, as_grey = True))
        names.append(i)
        k+=1    
    np.save('/content/imgs_test.npy', X_test)
    return X_test, names
        
    
    

In [0]:
def load_data():
  X_train = np.load('/content/img_train.npy')
  Y_train = np.load('/content/img_mask_train.npy')
  Y_test = np.load('/content/img_test.npy')
  return X_train, Y_train, Y_test



In [0]:
def resize_img(imgs):
  img_rows, img_cols = 96,96
  imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
  for i in range(imgs.shape[0]):
    imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)
  imgs_p = imgs_p[..., np.newaxis]
  return imgs_p

In [0]:
def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [0]:
def get_unet(input_shape):
    inputs = Input(input_shape)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

    return model

In [0]:
def train_model(X_train,Y_train,Y_test,names):
  X_train_re,Y_train_re,X_test_re = (resize_img(i) for i in [X_train,Y_train,X_test])
  X_train_re = X_train_re.astype(np.float64)
  X_train_re /= np.std(X_train_re) # scale X_train
  X_train_re -= np.mean(X_train_re) # center X_train
  Y_train_re = Y_train_re.astype(np.float64)
  Y_train_re /= 255. # scale Y_train [0,1]
  model = get_unet(X_train_re.shape[1:])
  reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.25, patience = 10,
                               verbose = 1, min_lr = 1e-12)
  
  model_check = ModelCheckpoint("Unet_model.h5", monitor='val_loss', verbose=0,
                                save_best_only=True, save_weights_only=True)
  
  model.fit(X_train_re, Y_train_re,batch_size = 32, epochs = 100 ,verbose = 2, callbacks = [reduce_lr, model_check],
           validation_split = 0.2, shuffle = True)
  
  X_test_re = resize_img(X_test)
  X_test_re = X_test.astype(np.float64)
  X_test_re /= np.std(X_test_re)
  X_test_re -= np.mean(X_test_re)
  
  model.load_weights('Unet_model.h5')
  Y_test = model.predict(X_test_re)
  
  os.mkdir(pred_img)
  for img, img_name in Y_test, names:
    image = (image[:, :, 0] * 255.).astype(np.uint8)
    imsave(('./pred_img'+ img_name + '_pred.png'), image)    

  
  
  

In [0]:
if __name__ == '__main__':
  os.chdir('./train')
  X_train, Y_train = create_data()
  os.chdir('/content/test')
  X_test, names = create_data()
  

In [0]:
train_model(X_train, Y_train, X_test, names)